In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/RobbieGeoghegan/AB_Testing/master/ab_data.csv")

In [3]:
df.shape

(294478, 5)

In [4]:
df.head(3)

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0


In [5]:
df.group.unique()

array(['control', 'treatment'], dtype=object)

In [6]:
df.landing_page.unique()

array(['old_page', 'new_page'], dtype=object)

In [7]:
df["group"].value_counts()

treatment    147276
control      147202
Name: group, dtype: int64

In [9]:
#some of the control group saw the new_page and some tretment group saw the old_page - delete these instances
mask1 = (df["group"] == "control") & (df["landing_page"] == "new_page")
index_to_drop1 = df[mask1].index
df = df.drop(index_to_drop1)

mask2 = (df["group"] == "treatment") & (df["landing_page"] == "old_page")
index_to_drop2 = df[mask2].index
df = df.drop(index_to_drop2)

print(df.shape)
df["group"].value_counts()

(290585, 5)


treatment    145311
control      145274
Name: group, dtype: int64

In [10]:
#Check how many duplicated users exist
print(df["user_id"].count())
print(df["user_id"].nunique())

290585
290584


In [11]:
#drop duplicated users
df.drop_duplicates(subset ='user_id',keep ='first',inplace = True)

In [12]:
df.shape

(290584, 5)

In [13]:
#Show the % split between users who saw new vs old page
#Calculate pooled probability
mask = (df["group"] == "control")
conversions_control = df["converted"][mask].sum()
total_users_control = df["converted"][mask].count()

In [14]:
mask = (df["group"] == "treatment")
conversions_treatment = df["converted"][mask].sum()
total_users_treatment = df["converted"][mask].count()

In [15]:
print("Split of control users who saw old page vs treatment users who saw new page: ", 
          round(total_users_control / df["converted"].count() * 100, 2), "% ",
          round((total_users_treatment / df["converted"].count()) * 100, 2), "%")

#count number of users who converted in each group
print("Number of control users who converted on old page: ", conversions_control)
print("Percentage of control users who converted: ", round((conversions_control / total_users_control) * 100, 2), "%")

mask = (df["group"] == "treatment")
print("Number of treatment users who converted on new page: ", conversions_treatment)
print("Percentage of treatment users who converted: ", round((conversions_treatment/ total_users_treatment) * 100, 2), "%")

Split of control users who saw old page vs treatment users who saw new page:  49.99 %  50.01 %
Number of control users who converted on old page:  17489
Percentage of control users who converted:  12.04 %
Number of treatment users who converted on new page:  17264
Percentage of treatment users who converted:  11.88 %


In [17]:
import math
import statsmodels.stats.api as sms
import scipy.stats as st

#Check what sample size is required
baseline_rate = conversions_control / total_users_control
practical_significance = 0.01 #user defined
confidence_level = 0.05 #user defined, for a 95% confidence interval
sensitivity = 0.8 #user defined

effect_size = sms.proportion_effectsize(baseline_rate, baseline_rate + practical_significance)
sample_size = sms.NormalIndPower().solve_power(effect_size = effect_size, power = sensitivity, 
                                               alpha = confidence_level, ratio=1)
print("Required sample size: ", round(sample_size), " per group")

Required sample size:  17209  per group
